# DATA CLEANING

## Mô Tả

Làm sạch và xét tính hợp lý để lọc dữ liệu.

## Script

### Using

In [1]:
import pandas as pd

### Contract Fields

In [2]:
CN_INC = 'Income'  # income column name
CN_AGE = 'Age'  # age column name
CN_EXP = 'Experience'  # experience column name
CN_MS = 'Married/Single'  # married or single column name
CN_HO = 'House_Ownership'  # house ownership column name
CN_CO = 'Car_Ownership'  # car ownership column name
CN_PRO = 'Profession'  # profession column name
CN_STA = 'STATE'  # state column name
CN_RF = 'Risk_Flag'  # risk flag column name

In [3]:
MS_VLD_VAS = ['married', 'single']  # married or single valid values
HO_VLD_VAS = ['rented', 'norent noown', 'owned']  # house ownership valid values
CO_VLD_VAS = ['no', 'yes']  # car ownership valid values
STA_VLD_VAS = [
    'Andhra Pradesh',
    'Arunachal Pradesh',
    'Assam',
    'Bihar',
    'Chhattisgarh',
    'Goa',
    'Gujarat',
    'Haryana',
    'Himachal Pradesh',
    'Jharkhand',
    'Karnataka',
    'Kerala',
    'Madhya Pradesh',
    'Maharashtra',
    'Manipur',
    'Meghalaya',
    'Mizoram',
    'Nagaland',
    'Odisha',
    'Punjab',
    'Rajasthan',
    'Sikkim',
    'Tamil Nadu',
    'Telangana',
    'Tripura',
    'Uttar Pradesh',
    'Uttarakhand',
    'West Bengal',
    'Andaman and Nicobar',
    'Chandigarh',
    'Dadra and Nagar Haveli and Daman and Diu',
    'Delhi',
    'Jammu and Kashmir',
    'Lakshadweep',
    'Ladakh',
    'Puducherry'
]  # state valid values
RF_VLD_VAS = [1, 0]  # risk flag valid values

### Contract Methods

In [4]:
# Refactor
def refactor(s):
    return s.str.strip().str.replace('_', ' ').str.split().str.join(' ').str.lower()

In [5]:
# Try parse to number
def try_prs_int(s):
    return pd.to_numeric(s, errors='coerce').astype('Int64')

### Processing

In [6]:
df = pd.read_excel('India_bank_loan_risk.xlsx')

In [7]:
income = try_prs_int(df[CN_INC])
age = try_prs_int(df[CN_AGE])
experience = try_prs_int(df[CN_EXP])
marr_sngl = refactor(df[CN_MS])
hse_or = refactor(df[CN_HO])
car_or = refactor(df[CN_CO])
profession = refactor(df[CN_PRO]).str.capitalize()
state = refactor(df[CN_STA]).str.title().str.replace(' And ', ' and ')
risk_flag = try_prs_int(df[CN_RF])

In [8]:
valids = (income.notna()
          & age.notna()
          & experience.notna()
          & marr_sngl.notna()
          & hse_or.notna()
          & car_or.notna()
          & profession.notna()
          & state.notna()
          & risk_flag.notna()
          & ~marr_sngl.eq('')
          & ~hse_or.eq('')
          & ~car_or.eq('')
          & ~profession.eq('')
          & ~state.eq('')
          & income.ge(0)
          & age.ge(18)
          & experience.ge(0)
          & ((age - experience).ge(18))
          & marr_sngl.isin(MS_VLD_VAS)
          & hse_or.isin(HO_VLD_VAS)
          & car_or.isin(CO_VLD_VAS)
          & state.isin(STA_VLD_VAS)
          & risk_flag.isin(RF_VLD_VAS))
invalids = ~valids

In [9]:
income = income.loc[valids]
age = age.loc[valids]
experience = experience.loc[valids]
marr_sngl = marr_sngl.loc[valids]
hse_or = hse_or.loc[valids]
car_or = car_or.loc[valids]
profession = profession.loc[valids]
state = state.loc[valids]
risk_flag = risk_flag.loc[valids]

In [10]:
pd.DataFrame({
    'Income': income,
    'Age': age,
    'Experience': experience,
    'Married/Single': marr_sngl,
    'House Ownership': hse_or,
    'Car Ownership': car_or,
    'Profession': profession,
    'State': state,
    'Risk Flag': risk_flag
}).to_csv('data_valid.csv')
df.loc[invalids].to_csv('data_invalid.csv')